In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets
import unittest

In [95]:
class Laguerre:
    def __init__(self,beta,sigma):
        self.beta = beta
        self.sigma = sigma
        
    @property
    def beta(self):
        return self._beta
    
    @beta.setter
    def beta(self, value):
        self._beta = value
    
    @property 
    def sigma(self):
        return self._sigma
    
    @sigma.setter
    def sigma(self, value):
        self._sigma = value
        
        
    def laguerre_polynom(self, t, n):
        if self.beta<0 or self.beta>self.sigma:
            raise ValueError("Wrong parameters")

        l_prev = np.sqrt(self.sigma) * (1 - self.sigma * t) * np.exp(-self.beta * t / 2)
        l_prev2 = np.sqrt(self.sigma) * np.exp(-self.beta * t / 2)
        if n == 1:
            return l_prev
        if n == 0:
            return l_prev2

        for i in range(2, n + 1):
            temp = l_prev
            l_prev = (2 * i - 1 - self.sigma * t) * l_prev / i - (i - 1) * l_prev2 / i
            l_prev2 = temp

        return l_prev
    
    def laguerre_polynom_tabulation(self,n, t_max, t_step = 0.1):
        t_values = np.arange(0, t_max, t_step)

        laguerre_values = [self.laguerre_polynom(t=i,n=n) for i in t_values]

        result = pd.DataFrame(data={'t': t_values, f'L_{n}': laguerre_values})

        return result

    def experiment(self, n_max = 20, eps = 1e-3, t_max = 100, t_points = 1000):
        N = range(0, n_max + 1)
        suitable_t = None

        for t in np.linspace(0, t_max, t_points):
            if all(abs(self.laguerre_polynom(t=t, n=n)) <= eps for n in N):
                suitable_t = t
                break

        result = pd.DataFrame(data={'n': N, 'L_n':
                                    [self.laguerre_polynom(t=suitable_t, n=n) for n in N]})

        return suitable_t, result

    def integral(f, a, b, int_points = 1000):
        x = np.linspace(a, b, int_points)
        s = sum([f(i) for i in x])
        return s * abs(b - a) / int_points

    def laguerre_transform(self, f, n_max, int_points = 10000):
        def integrand(t):
            alpha = self.sigma - self.beta
            return f(t) * self.laguerre_polynom(t=t, n=n_max) * np.exp(-alpha * t)

        t_max = self.experiment(n_max=n_max)[0]

        return integral(integrand, 0, t_max, int_points)

    def laguerre_transform_tabulation(self, f, n_max, int_points = 10000):
        N = range(0, n_max+1)
        return pd.DataFrame({'n': N, f'L_n': [self.laguerre_transform(f=f, n_max=n) for n in N]})

    def inverse_laguerre_transform(h,t):

        return sum([h[k] * self.laguerre_polynom(t=t, n=k) for k in range(0, len(h))])
    
    def widget_1(a):
        wd1 = ipywidgets.IntSlider(min=1,max=40,value=1)
        wd2 = ipywidgets.IntSlider(min=1,max=40,value=1)

        def interactive_laguerre_polynom_tabulation(n, t_max):
            display(a.laguerre_polynom_tabulation(n=n, t_max=t_max))
            
        ipywidgets.interact(interactive_laguerre_polynom_tabulation, n=wd1, t_max=wd2)

In [96]:
laguerre_instance = Laguerre(2.0, 4.0)
Laguerre.widget_1(laguerre_instance)

interactive(children=(IntSlider(value=1, description='n', max=40, min=1), IntSlider(value=1, description='t_ma…

In [102]:
class Laguerre_plot:
    def __init__(self, laguerre):
        self.laguerre = laguerre
    
    @property 
    def laguerre(self):
        return self._laguerre
        
    @laguerre.setter
    def laguerre(self, value):
        self._laguerre = value

    def laguerre_polynom_plot(self, t_max, n_max, t_step=0.01):
        plt.figure(figsize=(20, 15))

        for n in range(0, n_max + 1):
            l_n_tabulation = self.laguerre.laguerre_polynom_tabulation(n=n, t_max=t_max, t_step=t_step)
            plt.plot(l_n_tabulation['t'], l_n_tabulation[f'L_{n}'], label=f'L_{n}')

        plt.title('Laguerre Polynomials')
        plt.xlabel('t')
        plt.ylabel('L_n(t)')
        plt.legend()
        plt.grid()
        plt.show()

    def laguerre_transform_plot(self, f, n_max, t_max=np.pi * 2, t_step=0.01, int_points=10000):
        h = self.laguerre.laguerre_transform_tabulation(f=f, n_max=n_max)['L_n'].tolist()

        T = np.arange(0, t_max, t_step)
        inverse_laguerre_transform_tabulation = pd.DataFrame({'t': T, 'h': [self.laguerre.laguerre_transform(h=h, n_max=n_max, t=t) for t in T]})

        plt.figure(figsize=(10, 10))
        plt.plot(inverse_laguerre_transform_tabulation['t'], inverse_laguerre_transform_tabulation['h'])
        plt.title('Inverse Laguerre Transform')
        plt.xlabel('t')
        plt.ylabel('func(t)')
        plt.grid()
        plt.show()
    
    def print_widget(self):
        wd1 = ipywidgets.IntSlider(min=1,max=40,value=1)
        wd2 = ipywidgets.IntSlider(min=1,max=40,value=1)
        ipywidgets.interact(self.laguerre_polynom_plot, t_max=wd1, n_max=wd2, t_step = ipywidgets.fixed(0.001), beta= ipywidgets.fixed(2.0), sigma= ipywidgets.fixed(4.0))

In [103]:
laguerre_instance = Laguerre(2.0, 4.0) 
laguerre_plot_instance = Laguerre_plot(laguerre_instance)
laguerre_plot_instance.print_widget()

interactive(children=(IntSlider(value=1, description='t_max', max=40, min=1), IntSlider(value=1, description='…

In [116]:
class TestLaguerre(unittest.TestCase):
    def setUp(self):
        self.laguerre = Laguerre(beta=2.0, sigma=4.0)
        self.initial_beta = 2.0
        self.initial_sigma = 4.0
        self.initial_laguerre = Laguerre(beta=self.initial_beta, sigma=self.initial_sigma)
    
    def test_beta_setter_valid(self):
        self.laguerre.beta = 3.0
        self.assertEqual(self.laguerre.beta, 3.0)

    def test_beta_setter_invalid(self):
        with self.assertRaises(ValueError):
            self.laguerre.beta = -1.0

    def test_sigma_setter_valid(self):
        self.laguerre.sigma = 5.0
        self.assertEqual(self.laguerre.sigma, 5.0)

    def test_sigma_setter_invalid(self):
        with self.assertRaises(ValueError):
            self.laguerre.sigma = -1.0

    def test_beta_property(self):
        self.assertEqual(self.initial_laguerre.beta, self.initial_beta)
        
        new_valid_beta = 3.0
        self.initial_laguerre.beta = new_valid_beta
        self.assertEqual(self.initial_laguerre.beta, new_valid_beta)

        with self.assertRaises(ValueError):
            self.initial_laguerre.beta = -1.0

    def test_sigma_property(self):
        self.assertEqual(self.initial_laguerre.sigma, self.initial_sigma)

        new_valid_sigma = 5.0
        self.initial_laguerre.sigma = new_valid_sigma
        self.assertEqual(self.initial_laguerre.sigma, new_valid_sigma)

        with self.assertRaises(ValueError):
            self.initial_laguerre.sigma = -1.0
        
    def test_laguerre_polynom(self):
        result_n0 = self.laguerre.laguerre_polynom(0.5, 0)
        expected_n0 = np.sqrt(self.laguerre.sigma) * np.exp(-self.laguerre.beta * 0.5 / 2)
        self.assertAlmostEqual(result_n0, expected_n0)

        result_n1 = self.laguerre.laguerre_polynom(0.5, 1)
        expected_n1 = np.sqrt(self.laguerre.sigma) * (1 - self.laguerre.sigma * 0.5) * np.exp(-self.laguerre.beta * 0.5 / 2)
        self.assertAlmostEqual(result_n1, expected_n1)
        
    def test_laguerre_polynom_tabulation(self):
        result = self.laguerre.laguerre_polynom_tabulation(3, 5, 0.1)
        self.assertIsInstance(result, pd.DataFrame)
        self.assertEqual(len(result), int(5 / 0.1))

    def test_experiment(self):
        result_t, result_df = self.laguerre.experiment(n_max=10, eps=1e-3, t_max=100, t_points=1000)
        self.assertIsInstance(result_t, float)
        self.assertIsInstance(result_df, pd.DataFrame)
        self.assertTrue('n' in result_df.columns)
        self.assertTrue('L_n' in result_df.columns)

    def test_integral(self):
        def f(x):
            return x ** 2

        result = Laguerre.integral(f, 0, 1, int_points=1000)
        expected_result = 0.3335001668335001
        self.assertAlmostEqual(result, expected_result)   
            
unittest.main(argv=[''], exit=False)

..........
----------------------------------------------------------------------
Ran 10 tests in 0.024s

OK
